In [1]:
!pip install tensorflow
!pip install tensorflow_hub
!pip install tensorflow_datasets
!pip install panda
!pip install pandas


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import numpy as np
import time

import PIL.Image as Image
import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

import tensorflow_datasets as tfds
import baggage_dataset_images_only

import datetime

%load_ext tensorboard
%load_ext autoreload
%autoreload 2

2023-09-22 18:15:28.670306: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
ds_split, ds_info = tfds.load("MyDatasetBuilder", split=['train[:20%]', 'train[20%:]'], with_info=True, batch_size=32)

val_ds = ds_split[0]
train_ds = ds_split[1]

2023-09-22 18:15:30.570237: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-22 18:15:30.570275: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: dcc2c895a6fb
2023-09-22 18:15:30.570281: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: dcc2c895a6fb
2023-09-22 18:15:30.570352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: NOT_FOUND: was unable to find libcuda.so DSO loaded into this program
2023-09-22 18:15:30.570376: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 525.125.6


In [4]:
#train_ds = train_ds.map(lambda x: tf.image.resize(x, (224,224)))
#val_ds = val_ds.map(lambda x: tf.image.resize(x, (224,224)))

#normalization_layer = tf.keras.layers.Rescaling(1./255)
#train_ds_rn = train_ds_r.map(lambda x,y: (normalization_layer(x),y)) # Where x—images, y—labels.
#val_ds_rn = val_ds_r.map(lambda x,y: (normalization_layer(x),y)) # Where x—images, y—labels.

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [5]:
detector_model = "https://tfhub.dev/tensorflow/ssd_mobilenet_v2/fpnlite_320x320/1"

detector_layer = hub.KerasLayer(
    detector_model,
    input_shape=(224,224,3),
    dtype=tf.uint8,
    trainable=False,
    output_key="detection_classes"
)

In [6]:
num_classes = len(['handbag', 'suitcase', 'stroller', 'backpack', 'golf_club', 'duffel_bag',])

model = tf.keras.Sequential([
  detector_layer,
  tf.keras.layers.Dense(num_classes, activation="sigmoid")
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (1, 100)                  0         
                                                                 
 dense (Dense)               (1, 6)                    606       
                                                                 
Total params: 606
Trainable params: 606
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1) # Enable histogram computation for every epoch.

In [8]:
NUM_EPOCHS = 10

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

Epoch 1/10


2023-09-22 18:15:41.793774: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [7]
	 [[{{node Placeholder/_4}}]]
2023-09-22 18:15:41.794207: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [7]
	 [[{{node Placeholder/_0}}]]


ValueError: in user code:

    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1052, in train_step
        self._validate_target_and_loss(y, loss)
    File "/usr/local/lib/python3.8/dist-packages/keras/engine/training.py", line 1006, in _validate_target_and_loss
        raise ValueError(

    ValueError: Target data is missing. Your model was compiled with loss=<keras.losses.SparseCategoricalCrossentropy object at 0x7fc529331760>, and therefore expects target data to be provided in `fit()`